In [11]:
#!usr/bin/env python
# -*- coding:utf-8 _*-

"""
@author: solinari
@file: test.py
@time: 2019/02/10
"""

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from uuid import uuid4

from pytorch.sklearn_lineregression import do_regression
from datamanager.collection import collection
from datamanager.collection import data_show
import torch
from torch.autograd import Variable
from torch_learning.tensorboardX_002 import TBwriter
from common.mongo.neteaseConn import NeteaseConn

def getStockList():
    client = NeteaseConn('D:/workspace/testproj/Conf/netease.conf')
    return client.getStocks()


def gen_training_data(code, start_date, end_date):
    def gen_filename():
        filehead = str(code) + '_'
        return filehead + str(uuid4())
    
    def gen_storage_data():
        pass
    
    c = collection.Collection()

    for result in c.getData(code="600007", start_date="2000-01-01", end_date="2010-12-31"):
            ret = do_regression(result, epochs=10000, thres=10,
                                DBSCAN_eps=4, DBSCAN_minsamples=3)
            for item in ret:
                w = item[0]
                b = item[1]
                score = item[4]

                # if line regression cov score < xx; drop this result
                if score < 0.2 or abs(w) < 0.01:
                    continue
                dataset = result[item[2]: item[3]]
                dataset_info = {}
                dataset_info['regression_w'] = w
                dataset_info['regression_b'] = b
                dataset_info['regression_score'] = score 
                print('w: ', w, 'score: ', score)
                _dt = []
                for _it in dataset:
                    _dt.append(_it['TCLOSE'])
                
                fig = plt.figure()
#                 plt.plot(np.array(list(range(0, len(_dt)))),
#                          np.array(_dt), label='label')

                _dt = np.array(_dt)
                data = pd.DataFrame(_dt)
#                 data.to_csv('test.csv')
    #             c = TBwriter('D:/workspace/test')
    #             c.plotline(x_data=np.array(list(range(0, len(_dt)))), y_data=np.array(_dt))

            # show = data_show.Plt()
            # show.load_data(data=dataset)
            # show.plot(w=w, b=b)
            # time.sleep(1)2


if __name__ == '__main__':
    datainfo = list()
#     codes = getStockList()
#     code = codes[100][0]
    gen_training_data(code=None, start_date=None, end_date=None)
    

w:  0.03032382621253214 score:  0.8253228643631337
None_780319db-6cba-4bc1-9550-cc9ddf0eaefe


<Figure size 432x288 with 0 Axes>